In [ ]:
!pip install -q transformers datasets timm accelerate

In [ ]:
from transformers import AutoImageProcessor, ConvNextForImageClassification, Trainer, TrainingArguments
from datasets import load_dataset, Dataset, DatasetDict
import torch
import numpy as np
from PIL import Image
import os

model_ckpt = "facebook/convnext-base-224"  # veya convnext-small, convnext-tiny

processor = AutoImageProcessor.from_pretrained(model_ckpt)
model = ConvNextForImageClassification.from_pretrained(model_ckpt, num_labels=9,ignore_mismatched_sizes=True)


In [ ]:
!pip install -q datasets

In [ ]:
!pip install pyheif pillow-heif

In [ ]:
pip install filetype

In [ ]:
import filetype
import os

def add_extension_with_filetype(file_path):
    if os.path.splitext(file_path)[1] == '':
        kind = filetype.guess(file_path)
        if kind:
            new_file = file_path + '.' + kind.extension
            os.rename(file_path, new_file)
            print(f"Uzantı eklendi: {file_path} -> {new_file}")
            return new_file
        else:
            print(f"Dosya tipi belirlenemedi: {file_path}")
            return file_path
    else:
        return file_path

# Klasörü dolaşalım
folder = '/content/drive/MyDrive/Raw'

for root, _, files in os.walk(folder):
    for file in files:
        full_path = os.path.join(root, file)
        add_extension_with_filetype(full_path)


In [ ]:
pip install --upgrade datasets fsspec

In [ ]:
from torchvision.datasets import ImageFolder
from torchvision import transforms
from torch.utils.data import DataLoader

data_dir = "/content/drive/MyDrive/Raw"

transform = transforms.Compose([
    transforms.Resize((224, 224)),  # ConvNeXt için 224x224 input boyutu
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])


dataset = ImageFolder(root=data_dir, transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

print(f"Toplam örnek sayısı: {len(dataset)}")
print(f"Etiketler: {dataset.classes}")


In [ ]:
from torch.utils.data import random_split

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


In [ ]:
from transformers import ConvNextForImageClassification, ConvNextFeatureExtractor
import torch
from PIL import Image
import requests

# Model ve feature extractor'ı indir
model_name = "facebook/convnext-tiny-224"

feature_extractor = ConvNextFeatureExtractor.from_pretrained(model_name)
model = ConvNextForImageClassification.from_pretrained(model_name)

model.eval()


In [ ]:
from torchvision.datasets import ImageFolder
from torchvision import transforms
from transformers import AutoImageProcessor

# Hugging Face'den model yükleniyor
model_ckpt = "facebook/convnext-tiny-224"
processor = AutoImageProcessor.from_pretrained(model_ckpt)

# Preprocessing için dönüşüm (transform)
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # ConvNext 224x224 bekliyor
    transforms.ToTensor(),
    transforms.Normalize(mean=processor.image_mean, std=processor.image_std)
])

# Dataset yükleme
dataset = ImageFolder(root="/content/drive/MyDrive/Raw", transform=transform)


In [ ]:
# Örnek görüntü ve label kontrolü
img, label = dataset[0]
print(img.shape)  # torch.Size([3, 224, 224])
print(label)      # 0 gibi sınıf indexi


modeli hazırla


In [ ]:
class_names = ['1. Potassium Deficiency', '2. Manganese Deficiency', '3. Magnesium Deficiency', '4. Black Scorch', '5. Leaf Spots', '6. Fusarium Wilt', '7. Rachis Blight', '8. Parlatoria Blanchardi', '9. Healthy sample']
label2id = {label: idx for idx, label in enumerate(class_names)}
id2label = {idx: label for label, idx in label2id.items()}


In [ ]:
from transformers import AutoModelForImageClassification

model = AutoModelForImageClassification.from_pretrained(
    "facebook/convnext-base-384",
    num_labels=9,
    label2id = {label: idx for idx, label in enumerate(dataset.class_to_idx)},
    id2label = {idx: label for label, idx in dataset.class_to_idx.items()},
    ignore_mismatched_sizes=True
)

In [ ]:
from transformers import AutoModelForImageClassification

model = AutoModelForImageClassification.from_pretrained(
    "facebook/convnext-base-384",
    num_labels=9,
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes=True
)

PyTorch ImageFolder → Hugging Face Dataset dönüşümü


In [ ]:
from torchvision.datasets import ImageFolder
from datasets import Dataset, Features, ClassLabel
from datasets.features import Image as HFDatasetImage
from PIL import Image as PILImage
from datasets import Value

data_dir = "/content/drive/MyDrive/Raw"

dataset = ImageFolder(root=data_dir, transform=None)

image_paths = []
labels = []

for img_path, label in dataset.samples:
    image_paths.append(img_path)
    labels.append(label)

dict_ds = {
    "image_path": image_paths,
    "label": labels
}

features = Features({
    "image_path": Value("string"),  # Burada dosya yolları
    "label": ClassLabel(names=list(set(labels)))
})


hf_dataset = Dataset.from_dict(dict_ds, features=features)

def safe_load_image_batch(example_batch):
    images = []
    for path in example_batch["image_path"]:
        try:
            img = PILImage.open(path).convert("RGB")
        except Exception as e:
            print(f"Hata dosyada: {path} -> {e}")
            img = None
        images.append(img)
    return {"image": images}

hf_dataset = hf_dataset.map(
    safe_load_image_batch,
    batched=True,
    batch_size=8,
    desc="Görüntüleri yükle"
)

def filter_none_images(example):
    return example["image"] is not None

hf_dataset = hf_dataset.filter(filter_none_images)

print(f"Temizlenmiş dataset hazır, toplam örnek sayısı: {len(hf_dataset)}")
print(hf_dataset.column_names)


In [ ]:
print(hf_dataset.features)


In [ ]:
def filter_none_images(example):
    return example["image"] is not None

hf_dataset = hf_dataset.filter(filter_none_images)
print(f"Temizlenmiş dataset, toplam örnek sayısı: {len(hf_dataset)}")


Eğitim Kodu (Trainer ile)


In [ ]:
!pip install -q --upgrade transformers

In [ ]:
!pip install -U transformers

In [ ]:
from transformers import TrainingArguments, Trainer, EarlyStoppingCallback, AutoImageProcessor, ConvNextForImageClassification
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from torchvision.datasets import ImageFolder
from datasets import Dataset
from PIL import Image
import json
import torch
from torchvision import transforms

# 1. Image processor ve model seçimi
processor = AutoImageProcessor.from_pretrained("facebook/convnext-tiny-224")

# Doğru sınıf sayısı ile model tanımı
data_dir = "/content/drive/MyDrive/Raw"
torch_dataset = ImageFolder(root=data_dir)
num_classes = len(torch_dataset.classes)

model = ConvNextForImageClassification.from_pretrained(
    "facebook/convnext-tiny-224",
    num_labels=num_classes,
    ignore_mismatched_sizes=True
)

# 2. Metrik fonksiyonu
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='weighted')
    acc = accuracy_score(labels, predictions)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

# 3. Dataset yükle
image_paths = [path for path, _ in torch_dataset.samples]
labels = [label for _, label in torch_dataset.samples]
dict_ds = {"image_path": image_paths, "label": labels}
hf_dataset = Dataset.from_dict(dict_ds)

# 4. Görselleri yükle (önceden resize edip tensor'a çeviriyoruz)
image_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

def load_images(example_batch):
    images = []
    for path in example_batch["image_path"]:
        img = Image.open(path).convert("RGB")
        img = image_transform(img)
        images.append(img)
    return {"pixel_values": images}

hf_dataset = hf_dataset.map(load_images, batched=True, num_proc=1)

# 5. Train/test ayır
split_dataset = hf_dataset.train_test_split(test_size=0.2)


# 6. transform fonksiyonu
def transform(example_batch):
    inputs = processor(images=example_batch["pixel_values"], return_tensors="pt")
    example_batch["pixel_values"] = list(inputs["pixel_values"])
    example_batch["labels"] = example_batch["label"]
    return example_batch

# Her iki subset'te ayrı ayrı transform uygula
train_dataset = split_dataset["train"].map(transform, batched=True)
test_dataset = split_dataset["test"].map(transform, batched=True)

encoded_dataset = split_dataset.map(transform, batched=True)

# 7. Eğitim argümanları
training_args = TrainingArguments(
    output_dir="./convnext-results",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=10,
    save_steps=100,
    eval_steps=100,
    logging_steps=10,
    learning_rate=2e-5,
    remove_unused_columns=False,
    save_total_limit=1,
    push_to_hub=False,
    report_to="none",
    fp16=True  # Colab için daha az bellek kullanımı
)
#data collator
def data_collator(features):
    pixel_values = torch.stack([f["pixel_values"] for f in features])
    labels = torch.tensor([f["labels"] for f in features])
    return {"pixel_values": pixel_values, "labels": labels}

# 8. Trainer nesnesi (custom data collator ile!)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# 9. Eğitimi başlat
train_output = trainer.train()

# 10. Metrikleri kaydet
with open("train_metrics.json", "w") as f:
    json.dump(train_output.metrics, f, indent=4)

eval_metrics = trainer.evaluate()
with open("eval_metrics.json", "w") as f:
    json.dump(eval_metrics, f, indent=4)

# 11. Modeli kaydet
trainer.save_model("./convnext-model")
processor.save_pretrained("./convnext-model")


In [ ]:
pip install transformers datasets torchvision scikit-learn

In [ ]:
import os
print(os.listdir())  # Bulunduğun dizindeki klasör ve dosyaları göster


In [ ]:
pip install --upgrade fsspec


In [ ]:
from torchvision.datasets import ImageFolder
from torchvision import transforms
from torch.utils.data import DataLoader

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

dataset = ImageFolder(root="/content/drive/MyDrive/Raw", transform=transform)
dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

print(dataset.class_to_idx)


In [ ]:
import shutil
shutil.copytree("/content/drive/MyDrive/Raw", "/tmp/Raw")


In [ ]:
import torch
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision import transforms
from transformers import ConvNextForImageClassification, ConvNextImageProcessor, Trainer, TrainingArguments

# 1. Veri yolu
data_dir = "/content/drive/MyDrive/Raw" 

# 2. Görsel transformasyonları
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# 3. Dataset ve split
dataset = ImageFolder(root=data_dir, transform=transform)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_ds, val_ds = torch.utils.data.random_split(dataset, [train_size, val_size])

# 4. Processor ve model
processor = ConvNextImageProcessor.from_pretrained("facebook/convnext-tiny-224")
model = ConvNextForImageClassification.from_pretrained(
    "facebook/convnext-tiny-224",
    num_labels=len(dataset.classes),
    id2label={i: c for i, c in enumerate(dataset.classes)},
    label2id={c: i for i, c in enumerate(dataset.classes)},
    ignore_mismatched_sizes=True,
)

# 5. Collate function — Trainer ile DataLoader
def collate_fn(batch):
    images, labels = zip(*batch)
    # processor, pil image listesi alır ve tensor döner
    pixel_values = processor(images=images, return_tensors="pt").pixel_values
    labels = torch.tensor(labels)
    return {"pixel_values": pixel_values, "labels": labels}

# 6. Trainer'ın içine DataLoader koymuyoruz direkt Dataset veriyoruz,
# Trainer kendi içinde batch ve collate yapıyor
# Bu yüzden train_ds ve val_ds kullanacağız, sadece collate_fn'u Trainer'a veriyoruz

training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=10,
    eval_steps=100,
    save_steps=100,
    logging_steps=10,
    learning_rate=2e-5,
    save_total_limit=1,
    push_to_hub=False,
    report_to="none",
    fp16=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    data_collator=collate_fn,
)

# 7. Eğitimi başlat
trainer.train()


yeni bir deneme


In [ ]:
!pip install --upgrade transformers


In [ ]:
pip install -U transformers


In [ ]:
import transformers
print(transformers.__version__)


In [ ]:
from transformers import (
    ConvNextForImageClassification,
    ConvNextImageProcessor,
    Trainer,
    TrainingArguments,
    EarlyStoppingCallback,
)
import torch
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision import transforms
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import json
from google.colab import drive

# 0. Google Drive'ı mount et (çalıştırdığında Drive'a erişim vereceksin)
drive.mount('/content/drive')

# 1. Veri yolu
data_dir = "/content/drive/MyDrive/Raw"  

# 2. Görsel transformasyonları
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# 3. Dataset ve split
dataset = ImageFolder(root=data_dir, transform=transform)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_ds, val_ds = torch.utils.data.random_split(dataset, [train_size, val_size])

# 4. Processor ve model
processor = ConvNextImageProcessor.from_pretrained("facebook/convnext-tiny-224")
model = ConvNextForImageClassification.from_pretrained(
    "facebook/convnext-tiny-224",
    num_labels=len(dataset.classes),
    id2label={i: c for i, c in enumerate(dataset.classes)},
    label2id={c: i for i, c in enumerate(dataset.classes)},
    ignore_mismatched_sizes=True,
)

# 5. Collate function (batch olarak işleme için)
def collate_fn(batch):
    images, labels = zip(*batch)
    pixel_values = processor(images=images, return_tensors="pt").pixel_values
    labels = torch.tensor(labels)
    return {"pixel_values": pixel_values, "labels": labels}

# 6. Metrik fonksiyonu (accuracy, f1, precision, recall)
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(axis=-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    return {
        "accuracy": acc,
        "f1": f1,
        "precision": precision,
        "recall": recall,
    }

# 7. TrainingArguments (Kaydetme, logging ve değerlendirme ayarları)
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/convnext-results3",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=100,
    save_steps=100,
    logging_steps=10,
    learning_rate=2e-5,
    save_total_limit=1,
    push_to_hub=False,
    report_to="none",
    fp16=True,
    # Early stopping için gerekenler
    load_best_model_at_end=True,
    metric_for_best_model="eval_accuracy",  
    eval_strategy="steps",       # 'epoch' da olabilir ama steps daha iyi erken durdurmada
    eval_steps=100,                    # Kaç adımda bir validation yapılacağı
)

# 8. Trainer nesnesi
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,  
    callbacks=[EarlyStoppingCallback(early_stopping_patience=7)],  # patience: kaç defa artmazsa dursun
)

# 9. Eğitimi başlat
train_output = trainer.train()

# 10. Metrikleri kaydet (Drive içine)
with open("/content/drive/MyDrive/train_metrics3.json", "w") as f:
    json.dump(train_output.metrics, f, indent=4)

eval_metrics = trainer.evaluate()
with open("/content/drive/MyDrive/eval_metrics3.json", "w") as f:
    json.dump(eval_metrics, f, indent=4)

# 11. Model ve processor'ü kaydet (Drive içine)
trainer.save_model("/content/drive/MyDrive/convnext-model3")
processor.save_pretrained("/content/drive/MyDrive/convnext-model3")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Some weights of ConvNextForImageClassification were not initialized from the model checkpoint at facebook/convnext-tiny-224 and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([9, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([9]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
It looks like you are trying to rescale already rescaled images. If the input images have pixel values between 0 and 1, set `do_rescale=False` to avoid rescaling them again.


Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
100,1.954400,1.982176,0.258278,0.169308,0.126020,0.258278
200,1.823200,1.843424,0.324503,0.246932,0.255279,0.324503
300,1.691800,1.730693,0.397351,0.351075,0.469063,0.397351
400,1.332300,1.675082,0.390728,0.346048,0.442424,0.390728
500,1.164800,1.509140,0.536424,0.524598,0.554514,0.536424
600,0.976100,1.445271,0.523179,0.494602,0.533465,0.523179
700,0.714200,1.441249,0.509934,0.508364,0.554134,0.509934
800,0.483000,1.287554,0.589404,0.569601,0.561559,0.589404
900,0.389500,1.291901,0.576159,0.566740,0.610091,0.576159
1000,0.240900,1.386017,0.562914,0.559899,0.632889,0.562914


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


['/content/drive/MyDrive/convnext-model3/preprocessor_config.json']

convnext-base-224 modeli


In [ ]:
from transformers import (
    ConvNextForImageClassification,
    ConvNextImageProcessor,
    Trainer,
    TrainingArguments,
    EarlyStoppingCallback,
)
import torch
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision import transforms
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import json
from google.colab import drive

# 0. Google Drive'ı mount et (Drive erişimi için)
drive.mount('/content/drive')

# 1. Veri yolu
data_dir = "/content/drive/MyDrive/Raw"  

# 2. Görsel transformasyonları
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# 3. Dataset ve split
dataset = ImageFolder(root=data_dir, transform=transform)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_ds, val_ds = torch.utils.data.random_split(dataset, [train_size, val_size])

# 4. Processor ve model (ConvNext Base versiyonu)
processor = ConvNextImageProcessor.from_pretrained("facebook/convnext-base-224")
model = ConvNextForImageClassification.from_pretrained(
    "facebook/convnext-base-224",
    num_labels=len(dataset.classes),
    id2label={i: c for i, c in enumerate(dataset.classes)},
    label2id={c: i for i, c in enumerate(dataset.classes)},
    ignore_mismatched_sizes=True,
)

# 5. Collate function (batch olarak işleme için)
def collate_fn(batch):
    images, labels = zip(*batch)
    pixel_values = processor(images=images, return_tensors="pt").pixel_values
    labels = torch.tensor(labels)
    return {"pixel_values": pixel_values, "labels": labels}

# 6. Metrik fonksiyonu (accuracy, f1, precision, recall)
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(axis=-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    return {
        "accuracy": acc,
        "f1": f1,
        "precision": precision,
        "recall": recall,
    }

# 7. TrainingArguments (Kaydetme, logging ve değerlendirme ayarları)
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/convnext-results-base",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=100,
    save_strategy="steps",
    save_steps=100,
    logging_steps=10,
    learning_rate=2e-5,
    save_total_limit=1,
    push_to_hub=False,
    report_to="none",
    fp16=True,                   # GPU varsa aç, yoksa False yap
    load_best_model_at_end=True,
    metric_for_best_model="eval_accuracy",
    eval_strategy="steps",
    eval_steps=100,
)

# 8. Trainer nesnesi
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=7)],
)

# 9. Eğitimi başlat
train_output = trainer.train(resume_from_checkpoint="/content/drive/MyDrive/convnext-results-base/checkpoint-2800")

# 10. Metrikleri kaydet
with open("/content/drive/MyDrive/train_metrics_base.json", "w") as f:
    json.dump(train_output.metrics, f, indent=4)

eval_metrics = trainer.evaluate()
with open("/content/drive/MyDrive/eval_metrics_base.json", "w") as f:
    json.dump(eval_metrics, f, indent=4)

# 11. Model ve processor'ü kaydet
trainer.save_model("/content/drive/MyDrive/convnext-model-base")
processor.save_pretrained("/content/drive/MyDrive/convnext-model-base")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Some weights of ConvNextForImageClassification were not initialized from the model checkpoint at facebook/convnext-base-224 and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([1000, 1024]) in the checkpoint and torch.Size([9, 1024]) in the model instantiated
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([9]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
It looks like you are trying to rescale already rescaled images. If the input images have pixel values between 0 and 1, set `do_rescale=False` to avoid rescaling them again.
Didn't manage to set back the RNG states of the CUDA because of the following error:
 'cuda'
This won't yield the same results as if the training had not been interrupted.


Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
2900,0.167300,0.156538,0.960265,0.961462,0.966235,0.960265
3000,0.196000,0.146740,0.973510,0.974817,0.979296,0.973510
3100,0.014400,0.234100,0.953642,0.954394,0.961084,0.953642
3200,0.063700,0.309545,0.927152,0.927244,0.930778,0.927152
3300,0.005500,0.310953,0.927152,0.927388,0.931528,0.927152
3400,0.046600,0.322303,0.927152,0.927449,0.931734,0.927152
3500,0.008300,0.276023,0.933775,0.934355,0.939307,0.933775
3600,0.027800,0.322993,0.940397,0.941033,0.946114,0.940397
3700,0.002400,0.285741,0.933775,0.934355,0.939307,0.933775


['/content/drive/MyDrive/convnext-model-base/preprocessor_config.json']

In [ ]:
!pip uninstall torch torchvision torchaudio -y
!pip cache purge
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118


Found existing installation: torch 2.6.0+cu118
Uninstalling torch-2.6.0+cu118:
  Successfully uninstalled torch-2.6.0+cu118
Found existing installation: torchvision 0.21.0+cu124
Uninstalling torchvision-0.21.0+cu124:
  Successfully uninstalled torchvision-0.21.0+cu124
Found existing installation: torchaudio 2.6.0+cu124
Uninstalling torchaudio-2.6.0+cu124:
  Successfully uninstalled torchaudio-2.6.0+cu124
Files removed: 38
Looking in indexes: https://download.pytorch.org/whl/cu118
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 905.3/905.3 MB ? eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.7/155.7 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 46.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 55.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 64.9 MB/s eta 0:00:00
  Attempting uninstall: triton
    Found existing installation: triton 3.2.0
    Uninstalling triton-3.2.0:
      Successfully unin

In [ ]:
!pip install transformers==4.53.2


In [ ]:
import inspect
from transformers import TrainingArguments

print(inspect.signature(TrainingArguments.__init__))



(self, output_dir: Optional[str] = None, overwrite_output_dir: bool = False, do_train: bool = False, do_eval: bool = False, do_predict: bool = False, eval_strategy: Union[transformers.trainer_utils.IntervalStrategy, str] = 'no', prediction_loss_only: bool = False, per_device_train_batch_size: int = 8, per_device_eval_batch_size: int = 8, per_gpu_train_batch_size: Optional[int] = None, per_gpu_eval_batch_size: Optional[int] = None, gradient_accumulation_steps: int = 1, eval_accumulation_steps: Optional[int] = None, eval_delay: Optional[float] = 0, torch_empty_cache_steps: Optional[int] = None, learning_rate: float = 5e-05, weight_decay: float = 0.0, adam_beta1: float = 0.9, adam_beta2: float = 0.999, adam_epsilon: float = 1e-08, max_grad_norm: float = 1.0, num_train_epochs: float = 3.0, max_steps: int = -1, lr_scheduler_type: Union[transformers.trainer_utils.SchedulerType, str] = 'linear', lr_scheduler_kwargs: Union[dict[str, Any], str, NoneType] = <factory>, warmup_ratio: float = 0.0, 

Model Değerlendirme ve Test


In [ ]:
import torch
from transformers import Trainer, AutoFeatureExtractor, ConvNextForImageClassification
from sklearn.metrics import classification_report
import numpy as np
import os
from torchvision.datasets import ImageFolder
from torchvision import transforms
from google.colab import drive
from sklearn.utils.multiclass import unique_labels
from sklearn.metrics import accuracy_score

# WandB'yi kapatmak için:
os.environ["WANDB_DISABLED"] = "true"

# 1. Google Drive'ı mount et
drive.mount('/content/drive', force_remount=True)

# 2. Model ve Processor'ü yükle
model_path = "/content/drive/MyDrive/convnext-model3"
model = ConvNextForImageClassification.from_pretrained(model_path)
processor = AutoFeatureExtractor.from_pretrained(model_path)

# 3. Test dataset (Raw klasöründeki veriyi tekrar yükle ve split)
data_dir = "/content/drive/MyDrive/Raw"
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

dataset = ImageFolder(root=data_dir, transform=transform)
_, test_size = int(0.8 * len(dataset)), len(dataset) - int(0.8 * len(dataset))
_, test_ds = torch.utils.data.random_split(dataset, [int(0.8 * len(dataset)), test_size])

# 4. Collate function (modelin beklentisine göre)
def collate_fn(batch):
    images, labels = zip(*batch)
    pixel_values = processor(images=images, return_tensors="pt").pixel_values
    labels = torch.tensor(labels)
    return {"pixel_values": pixel_values, "labels": labels}

# 5. Trainer ayarları
from transformers import TrainingArguments
args = TrainingArguments(output_dir="./test-eval", per_device_eval_batch_size=8)

trainer = Trainer(
    model=model,
    args=args,
    data_collator=collate_fn
)

# 6. Test tahmini
outputs = trainer.predict(test_ds)
y_pred = np.argmax(outputs.predictions, axis=1)
y_true = outputs.label_ids

# 7. Sınıf isimleri
label_names = dataset.classes  

# 8. Raporu oluştur
active_labels = unique_labels(y_true, y_pred)
active_label_names = [label_names[i] for i in active_labels]

report = classification_report(y_true, y_pred, labels=active_labels, target_names=active_label_names, digits=4)
# Accuracy hesapla
accuracy = accuracy_score(y_true, y_pred)

# Raporun sonuna accuracy'yi ekle
report_with_accuracy = report + f"\nAccuracy: {accuracy:.4f}\n"

print(report_with_accuracy)

# 9. Google Drive'a kaydet
save_path = "/content/drive/MyDrive/model_sonuclari3/degerlendirme_raporu3.txt"
os.makedirs(os.path.dirname(save_path), exist_ok=True)

with open(save_path, "w", encoding="utf-8") as f:
    f.write(report_with_accuracy)

print(f"\nDeğerlendirme raporu başarıyla kaydedildi: {save_path}")


/usr/local/lib/python3.11/dist-packages/transformers/models/convnext/feature_extraction_convnext.py:30: FutureWarning: The class ConvNextFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ConvNextImageProcessor instead.
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Mounted at /content/drive


                          precision    recall  f1-score   support

 1. Potassium Deficiency     0.8800    1.0000    0.9362        22
 2. Manganese Deficiency     1.0000    1.0000    1.0000         1
 3. Magnesium Deficiency     0.9000    1.0000    0.9474         9
         4. Black Scorch     0.8667    0.9286    0.8966        14
           5. Leaf Spots     0.9333    1.0000    0.9655        14
        6. Fusarium Wilt     0.9091    0.9091    0.9091        33
        7. Rachis Blight     1.0000    0.8710    0.9310        31
8. Parlatoria Blanchardi     0.9286    0.9286    0.9286        14
       9. Healthy sample     1.0000    0.8462    0.9167        13

                accuracy                         0.9272       151
               macro avg     0.9353    0.9426    0.9368       151
            weighted avg     0.9315    0.9272    0.9270       151

Accuracy: 0.9272


✅ Değerlendirme raporu başarıyla kaydedildi: /content/drive/MyDrive/model_sonuclari3/degerlendirme_raporu3.txt
